# Theano implementation of HMM algorithms

This notebook implements some of the HMM algorithms using Theano. These may be helpful for incorporating them in the graphs of some other models in Theano.

## Discrete HMM

We start with a toy HMM example from [Rabiner's](http://www.ee.columbia.edu/~dpwe/e6820/papers/Rabiner89-hmm.pdf) paper.

Many improvements can be added to this (and may be added in the future):
  
  * everything should be moved to the log-domain - for computation stability. This is especially important for GPUs which usually have a lower floating-point precision. This example here works only for the smallest examples (a few observations).
  
  * continous density models - this discrete example cannot be applied to most practical examples.
  
  * custom topologies - this is a simple ergodic example. In practice, we'd want to be able to design a specific transition graph for a particular use.
  
  * combining models - real applications rely on being able to combine several models to be able to train them for specific problems. E.g. combining many tri-phone models for speech recognition. This should also include the ability to do state-tying.
  
  * better training algorithms - only Baum-Welch and the simplest gradient descent are demonstrated here. There are other methods that oculd work better/faster.
  
The example below will demonstrate the code using a similar notation and equations from Rabiner's paper. First we import the required stuff:

In [21]:
import numpy as np
import theano
import theano.tensor as T
import pickle
from collections import OrderedDict

### HMM class code

Here we create the HMM class. Everything is compiled in the constructor. We also provide methods for all the individual algorithms:

In [14]:
class DiscreteHMM:
    
    def __init__(self, N=3, M=4):        
        
        updates={}
                
        pi = theano.shared((np.ones(N)/N).astype(theano.config.floatX))
        a = theano.shared((np.ones((N,N))/(N*np.ones(N))).astype(theano.config.floatX))
        b = theano.shared((np.ones((N,M))/(N*np.ones(M))).astype(theano.config.floatX))
        
        self.pi=pi
        self.a=a
        self.b=b
        
        O = T.ivector()
        TT = O.shape[0]
        
        
        #forward algorithm:
        
        alpha0=pi*b[:,O[0]]
        
        alpha_scan,upd = theano.scan(fn=lambda O,alpha_p: T.dot(alpha_p,a)*b[:,O],
                               sequences=O[1:],
                               outputs_info=alpha0)
        
        updates.update(upd)
        
        alpha=T.concatenate((alpha0.reshape((1,N)),alpha_scan))                
        
        #backward algorithm:
        
        beta0=T.ones(N).astype(theano.config.floatX)
        
        beta_scan,upd = theano.scan(fn=lambda O,beta_p: T.dot(beta_p*b[:,O],a.T),
                               sequences=O[1:],
                               outputs_info=beta0,
                               go_backwards=True)
        updates.update(upd)
        
        beta=T.concatenate((beta_scan[::-1],beta0.reshape((1,N))))        
        
        #full model probability:
        
        full_prob = alpha_scan[-1].sum()        
        
        #forward-backward probabilities:
        
        gamma=alpha*beta/full_prob        
        
        #viterbi algorithm:
        
        def viterbi_rec_step(O, delta_p, phi_p):
            m=delta_p*a.T
            phi=m.argmax(axis=1)
            delta=m[T.arange(N),phi]*b[:,O]
            return delta,phi
        
        phi0=T.zeros(N).astype('int64')

        [delta_scan, phi_scan], upd = theano.scan(fn=viterbi_rec_step,
                                                  sequences=O[1:],
                                                  outputs_info=[alpha0,phi0])        
        
        updates.update(upd)
        
        QT=phi_scan[-1].argmax()        
        vite_prob = delta_scan[-1,QT]
        
        Q_scan, upd = theano.scan(fn=lambda phi, Q: phi[Q],
                             sequences=phi_scan,
                             outputs_info=QT,
                             go_backwards=True)
        
        updates.update(upd)
                                                  
        Q=T.concatenate((Q_scan[::-1],QT.reshape((1,))))
        
        #transition probabilities
        
        xi=alpha[:-1].reshape((TT-1,N,1))*a.reshape((1,N,N))*b[:,O[1:]].T.reshape((TT-1,1,N))*beta[1:].reshape((TT-1,1,N))/full_prob
        
        #expected values
        
        exp_pi=gamma[0]
        
        exp_a=xi.sum(axis=0)/gamma[:-1].sum(axis=0).reshape((N,1))
        
        exp_b_map, upd = theano.map(fn=lambda k: T.sum(gamma[T.eq(O,k).nonzero()],axis=0)/T.sum(gamma,axis=0), 
                         sequences=T.arange(M))
        
        updates.update(upd)
        
        exp_b = exp_b_map.T
        
        exp_err = T.concatenate(((pi-exp_pi).ravel(),(a-exp_a).ravel(),(b-exp_b).ravel()))
        
        exp_mean_err = T.mean(exp_err**2)
        
        #Baum-Welch updates:
        
        baum_welch_updates=OrderedDict()
        exp_updates={pi:exp_pi,a:exp_a,b:exp_b}
        baum_welch_updates.update(updates)
        baum_welch_updates.update(exp_updates)
        
        #Gradient descent:
        
        cost=-T.log(full_prob)
        
        pi_grad=T.grad(cost=cost,wrt=pi)
        a_grad=T.grad(cost=cost,wrt=a)
        b_grad=T.grad(cost=cost,wrt=b)
        
        lr=T.scalar()
        
        pi_upd=pi-lr*pi_grad
        norm_pi_upd=pi_upd/pi_upd.sum()
        
        a_upd=a-lr*a_grad
        norm_a_upd=a_upd/a_upd.sum(axis=1).T
        
        b_upd=b-lr*b_grad
        norm_b_upd=b_upd/b_upd.sum(axis=0)
        
        gd_updates=OrderedDict()
        grad_updates={pi:norm_pi_upd,
                      a:norm_a_upd,
                      b:norm_b_upd}
        gd_updates.update(updates)
        gd_updates.update(grad_updates)            
        
        #function definitions
        
        self.forward_fun = theano.function(inputs=[O], outputs=alpha, updates=updates)
        
        self.backward_fun = theano.function(inputs=[O], outputs=beta, updates=updates)
        
        self.full_prob_fun = theano.function(inputs=[O], outputs=full_prob, updates=updates)
        
        self.gamma_fun = theano.function(inputs=[O], outputs=gamma, updates=updates)
        
        self.viterbi_fun = theano.function(inputs=[O], outputs=[Q,vite_prob], updates=updates)
    
        self.xi_fun = theano.function(inputs=[O], outputs=xi, updates=updates)            
        
        self.baum_welch_fun = theano.function(inputs=[O], outputs=[full_prob,exp_mean_err], updates=baum_welch_updates)
        
        self.gd_fun = theano.function(inputs=[O,lr], outputs=cost, updates=gd_updates)
    
    def setModel(self,pi,a,b):
        
        self.pi.set_value(pi.astype(theano.config.floatX))
        self.a.set_value(a.astype(theano.config.floatX))
        self.b.set_value(b.astype(theano.config.floatX))
    
    def forward(self, O):
        
        return self.forward_fun(O.astype('int32')) 
    
    
    def backward(self, O):
        
        return self.backward_fun(O.astype('int32'))        
    
    def full_prob(self, O):
        
        return self.full_prob_fun(O.astype('int32'))
    
    def gamma(self, O):
        
        return self.gamma_fun(O.astype('int32'))
    
    def viterbi(self, O):
        
        return self.viterbi_fun(O.astype('int32'))
    
    def xi(self, O):
        
        return self.xi_fun(O.astype('int32'))
    
    def baum_welch(self,O):
        
        return self.baum_welch_fun(O.astype('int32'))
    
    def gradient_descent(self,O,lr=0.01):
        
        return self.gd_fun(O.astype('int32'),lr)
        
        
        

### Model creation

We can either use the default (all equally probable) or some other random values to begin with. Here we will read the model parameters from a file created in my other notebook. It will allow us to make sure all the calculations match the ones there:

In [23]:
with open('../data/hmm.pkl') as f:
    O,pi,a,b,N,M,Time=pickle.load(f)
    
print 'Number of states: {}'.format(N)
print 'Number of observation classes: {}'.format(M)
print 'Number of time steps: {}'.format(Time) #T is taken by theano.tensor
print 'Observation sequence: {}'.format(O)
print 'Priors: {}'.format(pi)
print 'Transition matrix:\n{}'.format(a)
print 'Observation probability matrix:\n{}'.format(b)

Number of states: 3
Number of observation classes: 4
Number of time steps: 5
Observation sequence: [3 1 1 2 0]
Priors: [ 0.78380037  0.07988522  0.13631441]
Transition matrix:
[[ 0.19762551  0.14294511  0.65942938]
 [ 0.35323748  0.24211448  0.40464804]
 [ 0.12763736  0.41033146  0.46203117]]
Observation probability matrix:
[[ 0.37536286  0.21435122  0.32037336  0.33995268]
 [ 0.28848115  0.35628436  0.14879559  0.65702006]
 [ 0.33615598  0.42936442  0.53083105  0.00302726]]


Here we will contruct the HMM object. The constructor needs to compile everything and since we have a few functions, it may take a little while:

In [24]:
%time hmm=DiscreteHMM(N=N,M=M)

#we can also set the model parameters
hmm.setModel(pi,a,b)

CPU times: user 17.7 s, sys: 172 ms, total: 17.8 s
Wall time: 17.8 s


### Algorithms

Let's test the methots now. You can compare the values with the ones from my other notebook:

In [26]:
print 'Forward probabilities:\n{}'.format(hmm.forward(O))
print 'Backward probabilities:\n{}'.format(hmm.backward(O))
print 'Full model probability: {}'.format(hmm.full_prob(O))
print 'Complete state probability:\n{}'.format(hmm.gamma(O))
seq,vite_prob=hmm.viterbi(O)
print 'Viterbi sequence: {} its probability {}'.format(seq,vite_prob)
print 'State transition probability:\n{}'.format(hmm.xi(O))

Forward probabilities:
[[ 0.26645502  0.05248619  0.00041266]
 [ 0.01527275  0.01815819  0.08464377]
 [ 0.00433764  0.01471865  0.0242707 ]
 [ 0.00293278  0.00210437  0.01063258]
 [ 0.00100599  0.00152653  0.00258775]]
Backward probabilities:
[[ 0.01634315  0.0144613   0.01584348]
 [ 0.04414593  0.04067532  0.04380064]
 [ 0.14111529  0.1194201   0.11332435]
 [ 0.33708936  0.33846256  0.32159775]
 [ 1.          1.          1.        ]]
Full model probability: 0.00512027181685
Complete state probability:
[[ 0.85048521  0.14823793  0.00127688]
 [ 0.13167854  0.14424822  0.72407317]
 [ 0.11954594  0.34328309  0.53717095]
 [ 0.1930774   0.13910387  0.66781867]
 [ 0.19647208  0.29813427  0.50539362]]
Viterbi sequence: [0 2 2 2 0] its probability 0.000175862107426
State transition probability:
[[[  9.73174050e-02   1.07802279e-01   6.45365417e-01]
  [  3.42637934e-02   3.59666944e-02   7.80074447e-02]
  [  9.73402493e-05   4.79248323e-04   7.00286706e-04]]

 [[  1.78306568e-02   1.81412734e-0

### Baum-Welch

We will run 15 iterations of the Baum-Welch EM reestimation here. We will also output the model probability (which should increase with each iteration) and also the mean difference between the model parameters and their expected values (which will decrease to 0 as the model converges on the optimum).

In [28]:
hmm.setModel(pi,a,b)
for i in range(30):    
    prob,exp_err=hmm.baum_welch(O)
    print 'Iteration #{} P={} delta_exp={}'.format(i+1,prob,exp_err)

Iteration #1 P=0.00512027181685 delta_exp=0.0247722174972
Iteration #2 P=0.00472553214058 delta_exp=0.00643846765161
Iteration #3 P=0.00895740184933 delta_exp=0.00366491661407
Iteration #4 P=0.0141975516453 delta_exp=0.000873240816873
Iteration #5 P=0.0171108879149 delta_exp=0.000643724633846
Iteration #6 P=0.0183900222182 delta_exp=0.00185267126653
Iteration #7 P=0.0206706672907 delta_exp=0.00560718541965
Iteration #8 P=0.026864964515 delta_exp=0.00610663974658
Iteration #9 P=0.038870152086 delta_exp=0.00184295291547
Iteration #10 P=0.0508226118982 delta_exp=0.000590678479057
Iteration #11 P=0.0574444383383 delta_exp=0.000154764056788
Iteration #12 P=0.0601356551051 delta_exp=3.64723564417e-05
Iteration #13 P=0.0613080821931 delta_exp=8.5957472038e-06
Iteration #14 P=0.0618683956563 delta_exp=2.08958635994e-06
Iteration #15 P=0.0621437616646 delta_exp=5.93567904161e-07
Iteration #16 P=0.0622817054391 delta_exp=2.46383166314e-07
Iteration #17 P=0.0623527467251 delta_exp=1.55892223574e-

### Gradient Descent

Since this is Theano, we can easily implement GD using the built-in *grad* method. The cost is set as the negative log of the complete model probability (the sum of the last step of forward probabilities). The parameters are updated by subtracting the gradient of the negative log likelihood multiplied by a learning rate. The updated values have to also be renormalized to keep the stochasticity of the parameters.

In [27]:
hmm.setModel(pi,a,b)
for i in range(50):    
    prob=hmm.gradient_descent(O,1)
    print 'Iteration #{} P={}'.format(i+1,np.exp(-prob))

Iteration #1 P=0.00512027274817
Iteration #2 P=0.00613689888269
Iteration #3 P=0.00783415138721
Iteration #4 P=0.00776180718094
Iteration #5 P=0.00900354515761
Iteration #6 P=0.00957148615271
Iteration #7 P=0.0107465116307
Iteration #8 P=0.0114053161815
Iteration #9 P=0.0121842995286
Iteration #10 P=0.0125104524195
Iteration #11 P=0.012768981047
Iteration #12 P=0.012803777121
Iteration #13 P=0.0129273673519
Iteration #14 P=0.0135532636195
Iteration #15 P=0.0156619008631
Iteration #16 P=0.0211016517133
Iteration #17 P=0.0307422261685
Iteration #18 P=0.0418569408357
Iteration #19 P=0.0502914786339
Iteration #20 P=0.0551136508584
Iteration #21 P=0.0574619658291
Iteration #22 P=0.0588321425021
Iteration #23 P=0.0598732866347
Iteration #24 P=0.0607092119753
Iteration #25 P=0.0613107830286
Iteration #26 P=0.0616954192519
Iteration #27 P=0.0619234070182
Iteration #28 P=0.0620619431138
Iteration #29 P=0.0621535144746
Iteration #30 P=0.0622175484896
Iteration #31 P=0.0622608475387
Iteration #32

Comparing this with the Baum-Welch should make it clear it's not that much better to do it this way. According to the Rabiner paper, they should be roughly equivalent, although the renormalization is implemented slightly differently there. 

Tuning the learning rate may help the convergence and one may also think about other techniques (momentum, dynamic learning rate, Newton's method, etc).

## Log model

*work-in-progess*

Log arithmetic hints: [here](https://github.com/UFAL-DSG/alex/blob/master/alex/ml/logarithmetic.py)

This is a copy of the code above, but keeping everythin in the log domain. We assume all the values (parameters, etc) are logarithms of what they usually are:

In [31]:
from pylearn2.expr.basic import log_sum_exp

def LogDot(a,b):
    return log_sum_exp(a + b)

def LogSum(a,axis=None):
    if axis is None:        
        a_max = a.max()
        return a_max + T.log((T.exp(a - a_max)).sum())
    
    shp = a.shape
    shp[axis] = 1
    a_max = a.max(axis=axis)
    s = T.log(T.exp(a - a_max.reshape(shp)).sum(axis=axis))
    lse = a_max + s
    return lse

def LogAdd(a,b):
    return T.log(T.exp(a)+T.exp(b))

def LogSub(a,b):
    return T.log(T.exp(a)-T.exp(b))

def LogMean(a):
    return LogSum(a)-T.log(a.shape[0])

class LogDiscreteHMM:
    
    def __init__(self, N=3, M=4):        
        
        updates={}
                
        pi = theano.shared((np.zeros(N)/N).astype(theano.config.floatX))
        a = theano.shared((np.zeros((N,N))/(N*np.ones(N))).astype(theano.config.floatX))
        b = theano.shared((np.zeros((N,M))/(N*np.ones(M))).astype(theano.config.floatX))
        
        self.pi=pi
        self.a=a
        self.b=b
        
        O = T.ivector()
        TT = O.shape[0]
        
        
        #forward algorithm:
        
        alpha0=pi+b[:,O[0]]
        
        alpha_scan,upd = theano.scan(fn=lambda O,alpha_p: LogDot(alpha_p,a)+b[:,O],
                               sequences=O[1:],
                               outputs_info=alpha0)
        
        updates.update(upd)
        
        alpha=T.concatenate((alpha0.reshape((1,N)),alpha_scan))                
        
        #backward algorithm:
        
        beta0=T.zeros(N).astype(theano.config.floatX)
        
        beta_scan,upd = theano.scan(fn=lambda O,beta_p: LogDot(beta_p+b[:,O],a.T),
                               sequences=O[1:],
                               outputs_info=beta0,
                               go_backwards=True)
        updates.update(upd)
        
        beta=T.concatenate((beta_scan[::-1],beta0.reshape((1,N))))        
        
        #full model probability:
        
        full_prob = LogSum(alpha_scan[-1])
        
        #forward-backward probabilities:
        
        gamma=alpha+beta-full_prob        
        
        #viterbi algorithm:
        
        def viterbi_rec_step(O, delta_p, phi_p):
            m=delta_p+a.T
            phi=m.argmax(axis=1)
            delta=m[T.arange(N),phi]+b[:,O]
            return delta,phi
        
        phi0=T.zeros(N).astype('int64')

        [delta_scan, phi_scan], upd = theano.scan(fn=viterbi_rec_step,
                                                  sequences=O[1:],
                                                  outputs_info=[alpha0,phi0])        
        
        updates.update(upd)
        
        QT=phi_scan[-1].argmax()        
        vite_prob = delta_scan[-1,QT]
        
        Q_scan, upd = theano.scan(fn=lambda phi, Q: phi[Q],
                             sequences=phi_scan,
                             outputs_info=QT,
                             go_backwards=True)
        
        updates.update(upd)
                                                  
        Q=T.concatenate((Q_scan[::-1],QT.reshape((1,))))
        
        #transition probabilities
        
        xi=alpha[:-1].reshape((TT-1,N,1))+a.reshape((1,N,N))+b[:,O[1:]].T.reshape((TT-1,1,N))+beta[1:].reshape((TT-1,1,N))-full_prob
        
        #expected values
        
        exp_pi=gamma[0]
        
        exp_a=LogSum(xi,axis=0)-LogSum(gamma[:-1],axis=0).reshape((N,1))
        
        exp_b_map, upd = theano.map(fn=lambda k: LogSum(gamma[T.eq(O,k).nonzero()],axis=0)-LogSum(gamma,axis=0), 
                         sequences=T.arange(M))
        
        updates.update(upd)
        
        exp_b = exp_b_map.T
        
        exp_err = T.concatenate((LogSub(pi,exp_pi).ravel(),LogSub(a,exp_a).ravel(),LogSub(b,exp_b).ravel()))
        
        exp_mean_err = LogMean(exp_err+2)
        
        #Baum-Welch updates:
        
        baum_welch_updates=OrderedDict()
        exp_updates={pi:exp_pi,a:exp_a,b:exp_b}
        baum_welch_updates.update(updates)
        baum_welch_updates.update(exp_updates)
        
        #Gradient descent:
        
        cost=-T.log(full_prob)
        
        pi_grad=T.grad(cost=cost,wrt=pi)
        a_grad=T.grad(cost=cost,wrt=a)
        b_grad=T.grad(cost=cost,wrt=b)
        
        lr=T.scalar()
        
        pi_upd=LogSub(pi,lr+pi_grad)
        norm_pi_upd=pi_upd-LogSum(pi_upd)
        
        a_upd=LogSub(a,lr+a_grad)
        norm_a_upd=a_upd-LogSum(a_upd,axis=1).T
        
        b_upd=LogSub(b,lr+b_grad)
        norm_b_upd=b_upd-LogSum(b_upd,axis=0)
        
        gd_updates=OrderedDict()
        grad_updates={pi:norm_pi_upd,
                      a:norm_a_upd,
                      b:norm_b_upd}
        gd_updates.update(updates)
        gd_updates.update(grad_updates)            
        
        #function definitions
        
        self.forward_fun = theano.function(inputs=[O], outputs=alpha, updates=updates)
        
        self.backward_fun = theano.function(inputs=[O], outputs=beta, updates=updates)
        
        self.full_prob_fun = theano.function(inputs=[O], outputs=full_prob, updates=updates)
        
        self.gamma_fun = theano.function(inputs=[O], outputs=gamma, updates=updates)
        
        self.viterbi_fun = theano.function(inputs=[O], outputs=[Q,vite_prob], updates=updates)
    
        self.xi_fun = theano.function(inputs=[O], outputs=xi, updates=updates)            
        
        self.baum_welch_fun = theano.function(inputs=[O], outputs=[full_prob,exp_mean_err], updates=baum_welch_updates)
        
        self.gd_fun = theano.function(inputs=[O,lr], outputs=cost, updates=gd_updates)
    
    def setModel(self,pi,a,b):
        
        self.pi.set_value(pi.astype(theano.config.floatX))
        self.a.set_value(a.astype(theano.config.floatX))
        self.b.set_value(b.astype(theano.config.floatX))
    
    def forward(self, O):
        
        return self.forward_fun(O.astype('int32')) 
    
    
    def backward(self, O):
        
        return self.backward_fun(O.astype('int32'))        
    
    def full_prob(self, O):
        
        return self.full_prob_fun(O.astype('int32'))
    
    def gamma(self, O):
        
        return self.gamma_fun(O.astype('int32'))
    
    def viterbi(self, O):
        
        return self.viterbi_fun(O.astype('int32'))
    
    def xi(self, O):
        
        return self.xi_fun(O.astype('int32'))
    
    def baum_welch(self,O):
        
        return self.baum_welch_fun(O.astype('int32'))
    
    def gradient_descent(self,O,lr=0.01):
        
        return self.gd_fun(O.astype('int32'),lr)        
        

In [33]:
loghmm=LogDiscreteHMM(N=N,M=M)

#we can also set the model parameters
loghmm.setModel(pi,a,b)

ValueError: When compiling the inner function of scan (the function called by scan in each of its iterations) the following error has been encountered: The initial state (`outputs_info` in scan nomenclature) of variable IncSubtensor{Set;:int64:}.0 (argument number 1) has 2 dimension(s), while the corresponding variable in the result of the inner function of scan (`fn`) has 0 dimension(s) (it should be one less than the initial state). For example, if the inner function of scan returns a vector of size d and scan uses the values of the previous time-step, then the initial state in scan should be a matrix of shape (1, d). The first dimension of this matrix corresponds to the number of previous time-steps that scan uses in each of its iterations. In order to solve this issue if the two varialbe currently have the same dimensionality, you can increase the dimensionality of the variable in the initial state of scan by using dimshuffle or shape_padleft. 